In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Nous commençons par charger le dataset créé

In [2]:
df = pd.read_csv("result.csv")

In [3]:
df["AVGLES"]=df["AVGLES"].replace(np.nan, 0)
df["AVGCLES"]=df["AVGCLES"].replace(np.nan, 0)
df

,TYPE,ARRIVAL_TIME,SERVING_TIME,WAIT_TIME,WAVGC_LES,LES,AVGLES,AVGCLES,SERVEURS,Q0,...,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26
0,5,3801.0,152.0,1.0,-1.000000,0.0,0.0,0.00,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,3877.0,311.0,1.0,-1.000000,0.0,0.0,0.00,1,0,...,0,0,0,0,0,0,0,0,0,0
2,5,4087.0,149.0,1.0,1.000000,1.0,1.0,1.00,2,0,...,0,0,0,0,0,0,0,0,0,0
3,5,4158.0,269.0,1.0,1.000000,1.0,1.0,1.00,5,0,...,0,0,0,0,0,0,0,0,0,0
4,5,4150.0,303.0,1.0,1.000000,1.0,1.0,1.00,4,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462939,5,46656.0,6.0,1.0,10.787741,50.0,5.9,3.21,2,0,...,0,0,0,0,0,0,0,0,0,0
1462940,2,46545.0,81.0,109.0,10.337674,1.0,69.4,4.06,5,0,...,0,0,0,0,0,0,0,0,0,0
1462941,2,46653.0,88.0,1.0,67.700640,1.0,69.4,32.04,2,0,...,0,0,0,0,0,0,0,0,0,0
1462942,20,46741.0,3.0,1.0,8.597853,38.0,4.7,4.09,2,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
type_counts = df['TYPE'].value_counts()
top_5_types = type_counts.nlargest(5).index
new_df = df[df['TYPE'].isin(top_5_types)].copy()

In [7]:
top_5_types

Int64Index([5, 20, 2, 11, 9], dtype='int64')

In [8]:
grouped_df = new_df.groupby("TYPE")
for group, subset_df in grouped_df:
    print(f"Dataframe pour le type {group}")
    print(subset_df)

Dataframe pour le type 2
         TYPE  ARRIVAL_TIME  SERVING_TIME  WAIT_TIME  WAVGC_LES  LES  AVGLES  \
5           2        4193.0         283.0        1.0   1.000000  1.0     1.0   
6           2        4106.0         392.0        1.0   1.000000  1.0     1.0   
8           2        3986.0         530.0        1.0  -1.000000  0.0     0.0   
9           2        4416.0         118.0        1.0   1.000000  1.0     1.0   
15          2        4683.0         191.0        1.0   1.000000  1.0     1.0   
...       ...           ...           ...        ...        ...  ...     ...   
1462926     2       44975.0          77.0        1.0  19.237645  1.0    95.5   
1462931     2       45949.0         185.0        1.0  15.590116  1.0    95.5   
1462932     2       46110.0          70.0        1.0  12.672093  1.0    95.5   
1462940     2       46545.0          81.0      109.0  10.337674  1.0    69.4   
1462941     2       46653.0          88.0        1.0  67.700640  1.0    69.4   

         AVGCL

### Creation des ANN pour chaque type

Nous commençons par créer un dataframe pour chacun des 5 services, on le convertit en numpy array, on normalise les données et on sépare les données en train et test

In [9]:
X1,y1 = df[df['TYPE']==top_5_types[0]].copy().drop(columns=["WAIT_TIME","TYPE"]).to_numpy(),df[df['TYPE']==top_5_types[0]].copy()["WAIT_TIME"].to_numpy()
X2,y2 = df[df['TYPE']==top_5_types[1]].copy().drop(columns=["WAIT_TIME","TYPE"]).to_numpy(),df[df['TYPE']==top_5_types[1]].copy()["WAIT_TIME"].to_numpy()
X3,y3 = df[df['TYPE']==top_5_types[2]].copy().drop(columns=["WAIT_TIME","TYPE"]).to_numpy(),df[df['TYPE']==top_5_types[2]].copy()["WAIT_TIME"].to_numpy()
X4,y4 = df[df['TYPE']==top_5_types[3]].copy().drop(columns=["WAIT_TIME","TYPE"]).to_numpy(),df[df['TYPE']==top_5_types[3]].copy()["WAIT_TIME"].to_numpy()
X5,y5 = df[df['TYPE']==top_5_types[4]].copy().drop(columns=["WAIT_TIME","TYPE"]).to_numpy(),df[df['TYPE']==top_5_types[4]].copy()["WAIT_TIME"].to_numpy()

In [10]:
sc = StandardScaler()
X1 = sc.fit_transform(X1)
X2 = sc.fit_transform(X2)
X3 = sc.fit_transform(X3)
X4 = sc.fit_transform(X4)
X5 = sc.fit_transform(X5)

In [11]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size=0.2,random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2,test_size=0.2,random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3,test_size=0.2,random_state=42)
X4_train, X4_test, y4_train, y4_test = train_test_split(X4,y4,test_size=0.2,random_state=42)
X5_train, X5_test, y5_train, y5_test = train_test_split(X5,y5,test_size=0.2,random_state=42)

Nous allons créer un l'architecture de l'ANN, entraîner et evaluer un modèle pour chaque service

**Type 5**

In [12]:
ann = Sequential([
    Dense(units=32,activation="relu",kernel_initializer="normal"),
    Dense(units=16,activation="relu",kernel_initializer="normal"),
    Dense(units=8,activation="relu",kernel_initializer="normal"),
    Dense(units=1,activation="linear")
])

adam = tf.keras.optimizers.Adam(learning_rate=0.01)

ann.compile(optimizer=adam,loss="mean_squared_error", metrics="mse")

In [13]:
ann.fit(X1_train,y1_train,batch_size=32,epochs=20)

Epoch 1/20
14498/14498 [==============================] - 48s 3ms/step - loss: 772.9396 - mse: 772.9396
Epoch 2/20
14498/14498 [==============================] - 40s 3ms/step - loss: 681.4412 - mse: 681.4412
Epoch 3/20
14498/14498 [==============================] - 39s 3ms/step - loss: 664.3702 - mse: 664.3702
Epoch 4/20
14498/14498 [==============================] - 39s 3ms/step - loss: 660.8918 - mse: 660.8918
Epoch 5/20
14498/14498 [==============================] - 40s 3ms/step - loss: 652.1268 - mse: 652.1268
Epoch 6/20
14498/14498 [==============================] - 41s 3ms/step - loss: 648.0016 - mse: 648.0016
Epoch 7/20
14498/14498 [==============================] - 40s 3ms/step - loss: 643.4029 - mse: 643.4029
Epoch 8/20
14498/14498 [==============================] - 41s 3ms/step - loss: 641.8510 - mse: 641.8510
Epoch 9/20
14498/14498 [==============================] - 39s 3ms/step - loss: 642.2570 - mse: 642.2570
Epoch 10/20
14498/14498 [==============================] - 38s 3

In [14]:
score5 = ann.evaluate(X1_test, y1_test)

3625/3625 [==============================] - 9s 2ms/step - loss: 594.5507 - mse: 594.5507


In [15]:
print(f"Le RMSE obtenu avec ANN pour le service de type 5 est {score5[1]**0.5}")

Le RMSE obtenu avec ANN pour le service de type 5 est 24.383410758440743


In [16]:
ann.save("models/service_type_5.h5")

C:\Users\Massamba\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Type 20**

In [17]:
ann = Sequential([
    Dense(units=32,activation="relu",kernel_initializer="normal"),
    Dense(units=16,activation="relu",kernel_initializer="normal"),
    Dense(units=8,activation="relu",kernel_initializer="normal"),
    Dense(units=1,activation="linear")
])

adam = tf.keras.optimizers.Adam(learning_rate=0.01)

ann.compile(optimizer=adam,loss="mean_squared_error", metrics="mse")

In [18]:
ann.fit(X2_train,y2_train,batch_size=32,epochs=20)

Epoch 1/20
7819/7819 [==============================] - 22s 3ms/step - loss: 1162.3860 - mse: 1162.3860
Epoch 2/20
7819/7819 [==============================] - 21s 3ms/step - loss: 1049.0941 - mse: 1049.0941
Epoch 3/20
7819/7819 [==============================] - 21s 3ms/step - loss: 1014.7167 - mse: 1014.7167
Epoch 4/20
7819/7819 [==============================] - 20s 3ms/step - loss: 1001.7532 - mse: 1001.7532
Epoch 5/20
7819/7819 [==============================] - 20s 3ms/step - loss: 990.5463 - mse: 990.5463
Epoch 6/20
7819/7819 [==============================] - 20s 3ms/step - loss: 983.9070 - mse: 983.9070
Epoch 7/20
7819/7819 [==============================] - 20s 3ms/step - loss: 971.2883 - mse: 971.2883
Epoch 8/20
7819/7819 [==============================] - 20s 3ms/step - loss: 972.5643 - mse: 972.5643
Epoch 9/20
7819/7819 [==============================] - 20s 3ms/step - loss: 969.3234 - mse: 969.3234
Epoch 10/20
7819/7819 [==============================] - 22s 3ms/step - lo

In [19]:
score20=ann.evaluate(X2_test, y2_test)

1955/1955 [==============================] - 6s 3ms/step - loss: 964.1425 - mse: 964.1425


In [20]:
print(f"Le RMSE obtenu avec ANN pour le service de type 20 est {score20[1]**0.5}")

Le RMSE obtenu avec ANN pour le service de type 20 est 31.05064340806302


In [20]:
ann.save("models/service_type_20.h5")

**Type 2**

In [21]:
ann = Sequential([
    Dense(units=32,activation="relu",kernel_initializer="normal"),
    Dense(units=16,activation="relu",kernel_initializer="normal"),
    Dense(units=8,activation="relu",kernel_initializer="normal"),
    Dense(units=1,activation="linear")
])

adam = tf.keras.optimizers.Adam(learning_rate=0.01)

ann.compile(optimizer=adam,loss="mean_squared_error", metrics="mse")

In [22]:
ann.fit(X3_train,y3_train,batch_size=32,epochs=20)

Epoch 1/20
6656/6656 [==============================] - 22s 3ms/step - loss: 1544.9410 - mse: 1544.9410
Epoch 2/20
6656/6656 [==============================] - 18s 3ms/step - loss: 1340.8369 - mse: 1340.8369
Epoch 3/20
6656/6656 [==============================] - 18s 3ms/step - loss: 1282.9648 - mse: 1282.9648
Epoch 4/20
6656/6656 [==============================] - 18s 3ms/step - loss: 1254.6453 - mse: 1254.6453
Epoch 5/20
6656/6656 [==============================] - 18s 3ms/step - loss: 1243.7030 - mse: 1243.7030
Epoch 6/20
6656/6656 [==============================] - 18s 3ms/step - loss: 1235.9298 - mse: 1235.9298
Epoch 7/20
6656/6656 [==============================] - 18s 3ms/step - loss: 1228.0486 - mse: 1228.0486
Epoch 8/20
6656/6656 [==============================] - 18s 3ms/step - loss: 1225.2321 - mse: 1225.2321
Epoch 9/20
6656/6656 [==============================] - 17s 3ms/step - loss: 1221.3356 - mse: 1221.3356
Epoch 10/20
6656/6656 [==============================] - 18s 3ms

In [23]:
score2=ann.evaluate(X3_test, y3_test)

1664/1664 [==============================] - 4s 2ms/step - loss: 1157.4689 - mse: 1157.4689


In [24]:
print(f"Le RMSE obtenu avec ANN pour le service de type 2 est {score2[1]**0.5}")

Le RMSE obtenu avec ANN pour le service de type 2 est 34.02159420236378


In [25]:
ann.save("models/service_type_2.h5")

**Type 11**

In [26]:
ann = Sequential([
    Dense(units=32,activation="relu",kernel_initializer="normal"),
    Dense(units=16,activation="relu",kernel_initializer="normal"),
    Dense(units=8,activation="relu",kernel_initializer="normal"),
    Dense(units=1,activation="linear")
])

adam = tf.keras.optimizers.Adam(learning_rate=0.01)

ann.compile(optimizer=adam,loss="mean_squared_error", metrics="mse")

In [27]:
ann.fit(X4_train,y4_train,batch_size=32,epochs=20)

Epoch 1/20
3702/3702 [==============================] - 15s 4ms/step - loss: 986.0132 - mse: 986.0132
Epoch 2/20
3702/3702 [==============================] - 14s 4ms/step - loss: 828.1088 - mse: 828.1088
Epoch 3/20
3702/3702 [==============================] - 12s 3ms/step - loss: 812.5526 - mse: 812.5526
Epoch 4/20
3702/3702 [==============================] - 10s 3ms/step - loss: 793.8769 - mse: 793.8769
Epoch 5/20
3702/3702 [==============================] - 10s 3ms/step - loss: 780.1984 - mse: 780.1984
Epoch 6/20
3702/3702 [==============================] - 11s 3ms/step - loss: 776.1179 - mse: 776.1179
Epoch 7/20
3702/3702 [==============================] - 11s 3ms/step - loss: 760.3142 - mse: 760.3142
Epoch 8/20
3702/3702 [==============================] - 12s 3ms/step - loss: 758.2438 - mse: 758.2438
Epoch 9/20
3702/3702 [==============================] - 11s 3ms/step - loss: 751.4370 - mse: 751.4370
Epoch 10/20
3702/3702 [==============================] - 10s 3ms/step - loss: 747.

In [28]:
score11=ann.evaluate(X4_test, y4_test)

926/926 [==============================] - 2s 2ms/step - loss: 773.4893 - mse: 773.4893


In [29]:
print(f"Le RMSE obtenu avec ANN pour le service de type 11 est {score11[1]**0.5}")

Le RMSE obtenu avec ANN pour le service de type 11 est 27.811674847310076


In [30]:
ann.save("models/service_type_11.h5")

**Type 9**

In [31]:
ann = Sequential([
    Dense(units=32,activation="relu",kernel_initializer="normal"),
    Dense(units=16,activation="relu",kernel_initializer="normal"),
    Dense(units=8,activation="relu",kernel_initializer="normal"),
    Dense(units=1,activation="linear")
])

adam = tf.keras.optimizers.Adam(learning_rate=0.01)

ann.compile(optimizer=adam,loss="mean_squared_error", metrics="mse")

In [32]:
ann.fit(X5_train,y5_train,batch_size=32,epochs=20)

Epoch 1/20
1615/1615 [==============================] - 5s 3ms/step - loss: 1599.3818 - mse: 1599.3818
Epoch 2/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1444.0708 - mse: 1444.0708
Epoch 3/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1414.8021 - mse: 1414.8021
Epoch 4/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1372.6879 - mse: 1372.6879
Epoch 5/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1346.3817 - mse: 1346.3817
Epoch 6/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1332.3955 - mse: 1332.3955
Epoch 7/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1319.5823 - mse: 1319.5823
Epoch 8/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1306.8307 - mse: 1306.8307
Epoch 9/20
1615/1615 [==============================] - 4s 3ms/step - loss: 1297.6462 - mse: 1297.6462
Epoch 10/20
1615/1615 [==============================] - 4s 3ms/step - lo

In [33]:
score9=ann.evaluate(X5_test, y5_test)

404/404 [==============================] - 1s 2ms/step - loss: 1199.9482 - mse: 1199.9482


In [34]:
print(f"Le RMSE obtenu avec ANN pour le service de type 9 est {score9[1]**0.5}")

Le RMSE obtenu avec ANN pour le service de type 9 est 34.64026908364743


In [35]:
ann.save("models/service_type_9.h5")

In [37]:
scores = {
    "2": score2[1]**0.5,
    "5": score5[1]**0.5,
    "9": score9[1]**0.5,
    "11": score11[1]**0.5,
    "20": score20[1]**0.5
}

**Résultats finaux**

In [38]:
for group, subset_df in grouped_df:
    print(f"Pour le service {group}, le RMSE pour l'ANN est {scores[str(group)]}\n")

Pour le service 2, le RMSE pour l'ANN est 34.02159420236378

Pour le service 5, le RMSE pour l'ANN est 24.383410758440743

Pour le service 9, le RMSE pour l'ANN est 34.64026908364743

Pour le service 11, le RMSE pour l'ANN est 27.811674847310076

Pour le service 20, le RMSE pour l'ANN est 31.05064340806302

